### Gate pass system using open cv as the main library

### Python Modules used

1. cv2: image processing, specifically for capturing and displaying video from a webcam and reading and writing images.
2. face_recognition: provides functions for encoding and recognizing faces in images.
3. pickle: saving the encoded faces with student IDs to a file.
4. os: listing the files in the specified folder path.
5. firebase_admin: initializing the Firebase app and accessing the Firebase database and storage.
6. credentials: loading the service account key file.
7. db: accessing the database and updating data.
8. storage: uploading images to Firebase Storage.
9. face_recognition : face recognition, encoding and comparing faces in images.
10. cvzone : drawing rectangles and text on images. 
11. datetime : working with dates and times.    
    
    
    

In [1]:
# importing the required modules for database setup
import cv2
import face_recognition
import cvzone
import numpy as np
from datetime import datetime
import firebase_admin
from firebase_admin import credentials,storage,db
import json
import cv2
import face_recognition
import pickle
import os
import json

### Firebase services

Initialize the Firebase app using a service account key JSON file and configures it to use the Firebase Realtime Database.
The service account key JSON file *("serviceAccountKey.json")* contains the credentials needed to authenticate the Firebase app.

In [2]:
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
'databaseURL': "https://gate-pass-system-c8536-default-rtdb.firebaseio.com/",
    'storageBucket': "gate-pass-system-c8536.appspot.com"
})
   

### Setting up the database
##### Assumptions made
1. For this gatepass system its assumed that the database already exists.
2. The college in which the system is to be used has already registered the students and staff.
3. To use this system the college/university provides the already setup database

**Note :** This database is setup just for demonstartion purposes of the system

### Structure of the database

Each student record stored in the json file has the following attributes
1. name: the name of the student
2. course: the course of the student
3. starting_year: the year in which the student started attending the university
4. total_entrance: the total number of times the student has entered the gate
5. standing: the academic standing of the student
6. year: the current academic year of the student
7. last_entrace_time: the date and time of the student's last attendance



In [3]:
# Load data from json file
with open('data.json') as f:
    data = json.load(f)
    


ref = db.reference('Students')

for key, value in data.items():
    ref.child(key).set(value)

### Handle images
Upload images from local repository to an the firebase

In [4]:
# Define folder path for images
folderPath = 'Images'

# Get list of image paths in folder
pathList = os.listdir(folderPath)
print(pathList)

# Initialize empty lists for images and student IDs
imgList = []
studentIds = []

# Loop through image paths
for path in pathList:
    # Read image using OpenCV's imread function and append to image list
    imgList.append(cv2.imread(os.path.join(folderPath, path)))
    
    # Extract student ID from image filename and append to student ID list
    studentIds.append(os.path.splitext(path)[0])

    # Upload image to Firebase Storage
    fileName = f'{folderPath}/{path}'
    bucket = storage.bucket()
    blob = bucket.blob(fileName)
    blob.upload_from_filename(fileName)

['0123.png', '0124.png', '0125.jpg', '0126.jpg', '0127.png', '0128.jpg', '0129.png', '0130.jpg', '0131.jpg', '963852.png']


### Function to encode faces in a list of images

In [5]:
def findEncodings(imagesList):
    encodeList = []
    for img in imagesList:
        # Convert image to RGB format using OpenCV's cvtColor function
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Encode face in image using face_recognition's face_encodings function
        encode = face_recognition.face_encodings(img)[0]
        
        # Append encoded face to list of encoded faces
        encodeList.append(encode)

    # Return list of encoded faces
    return encodeList



### Save the encoded faces with student IDs to file using pickle

In [6]:
# Encode faces in image list using findEncodings function
encodeListKnown = findEncodings(imgList)

# Create list of encoded faces with their corresponding student IDs
encodeListKnownWithIds = [encodeListKnown, studentIds]

# Print message indicating encoding is complete
print("Encoding Complete")

# Save encoded faces with student IDs to file using pickle
file = open("EncodeFile.p", 'wb')
pickle.dump(encodeListKnownWithIds, file)
file.close()

Encoding Complete


### Capture video using webcam and  load the images used for the UI

In [7]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

#Load the background image
imgBackground = cv2.imread('Resources/background.png')
 
# Importing the mode images into a list
folderModePath = 'Resources/Modes'
modePathList = os.listdir(folderModePath)
imgModeList = []
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath, path)))



### Load the encoding file

In [8]:
file = open('EncodeFile.p', 'rb')
encodeListKnownWithIds = pickle.load(file)
file.close()
encodeListKnown, studentIds = encodeListKnownWithIds

 
modeType = 0
counter = 0
id = -1
imgStudent = []

### Main loop of the program
The loop:
1. Reads frames from a video capture device.
2. Resizes and converts the frame to RGB.
3. Detects faces in the frame.
4. Encodes the faces and compares them to known faces. 

If a known face is detected, the code retrieves the student's information from a database, updates the attendance record, and displays the student's information and image on the screen.

In [9]:
while True:
       # Wait for a key press
    key = cv2.waitKey(1) & 0xFF
    
    # Check if the 'q' key was pressed
    if key == ord('q'):
        break

    success, img = cap.read()
 
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
 
    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)
 
    imgBackground[162:162 + 480, 55:55 + 640] = img
    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
 
    if faceCurFrame:
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
            # print("matches", matches)
            # print("faceDis", faceDis)
 
            matchIndex = np.argmin(faceDis)
            # print("Match Index", matchIndex)
 
            if matches[matchIndex]:
                # print("Known Face Detected")
                # print(studentIds[matchIndex])
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
                imgBackground = cvzone.cornerRect(imgBackground, bbox, rt=0)
                id = studentIds[matchIndex]
                if counter == 0:
                    cvzone.putTextRect(imgBackground, "Loading", (275, 400))
                    cv2.imshow("Gate Pass System", imgBackground)
                    cv2.waitKey(1)
                    counter = 1
                    modeType = 1
 
        if counter != 0:
 
            if counter == 1:
                # Get the Data
                studentInfo = db.reference(f'Students/{id}').get()
                print(studentInfo)
                # Get the Image from the storage
                blob = bucket.get_blob(f'Images/{id}.png')
                array = np.frombuffer(blob.download_as_string(), np.uint8)
                imgStudent = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
                # Update data of attendance
                datetimeObject = datetime.strptime(studentInfo['last_entrance_time'],
                                                   "%Y-%m-%d %H:%M:%S")
                secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
                print(secondsElapsed)
                if secondsElapsed > 30:
                    ref = db.reference(f'Students/{id}')
                    studentInfo['total_entrance'] += 1
                    ref.child('total_entrance').set(studentInfo['total_entrance'])
                    ref.child('last_entrance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                else:
                    modeType = 3
                    counter = 0
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
 
            if modeType != 3:
 
                if 10 < counter < 20:
                    modeType = 2
 
                imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
 
                if counter <= 10:
                    cv2.putText(imgBackground, str(studentInfo['total_entrance']), (861, 125),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['course']), (1006, 550),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(id), (1006, 493),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['standing']), (910, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['year']), (1025, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
                    cv2.putText(imgBackground, str(studentInfo['starting_year']), (1125, 625),
                                cv2.FONT_HERSHEY_COMPLEX, 0.6, (100, 100, 100), 1)
 
                    (w, h), _ = cv2.getTextSize(studentInfo['name'], cv2.FONT_HERSHEY_COMPLEX, 1, 1)
                    offset = (414 - w) // 2
                    cv2.putText(imgBackground, str(studentInfo['name']), (808 + offset, 445),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 50), 1)
 
                    imgBackground[175:175 + 216, 909:909 + 216] = imgStudent
 
                counter += 1
 
                if counter >= 20:
                    counter = 0
                    modeType = 0
                    studentInfo = []
                    imgStudent = []
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    else:
        modeType = 0
        counter = 0
    # cv2.imshow("Webcam", img)
    cv2.imshow("Gate Pass System", imgBackground)
    cv2.waitKey(1)

    
# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()

{'course': 'Computer Science', 'last_entrance_time': '2022-12-11 00:54:34', 'name': 'JacMwas', 'standing': 'G', 'starting_year': 2017, 'total_entrance': 0, 'year': 2}
8776942.056347
{'course': 'Computer Science', 'last_entrance_time': '2023-03-22 14:56:56', 'name': 'JacMwas', 'standing': 'G', 'starting_year': 2017, 'total_entrance': 1, 'year': 2}
17.840425
{'course': 'Computer Science', 'last_entrance_time': '2023-03-22 14:56:56', 'name': 'JacMwas', 'standing': 'G', 'starting_year': 2017, 'total_entrance': 1, 'year': 2}
19.407222
{'course': 'Computer Science', 'last_entrance_time': '2023-03-22 14:56:56', 'name': 'JacMwas', 'standing': 'G', 'starting_year': 2017, 'total_entrance': 1, 'year': 2}
21.070073
{'course': 'Computer Science', 'last_entrance_time': '2023-03-22 14:56:56', 'name': 'JacMwas', 'standing': 'G', 'starting_year': 2017, 'total_entrance': 1, 'year': 2}
22.644954
{'course': 'Computer Science', 'last_entrance_time': '2023-03-22 14:56:56', 'name': 'JacMwas', 'standing': 'G'